In [1]:
import  requests
import json
import pandas as pd  
import os 
import mimetypes
import time 


In [2]:
job_dir = "JobDesc"
resume_dir = "Resumes"

In [3]:
urls = {
    "upload_resume" : "https://avermass-cv-ranking.com/uploadfiles/resumes",
    "upload_jobs" : "https://avermass-cv-ranking.com/uploadfiles/jobbdesc",
    "similarity_ranking" : "https://avermass-cv-ranking.com/similarity-rankings"
}

In [4]:
def upload_resume(client_id , resume_path, max_tries = 10):
  payload={}
  mime_type, _ = mimetypes.guess_type(resume_path)

  # If the MIME type cannot be determined, set it to 'application/octet-stream'
  if mime_type is None:
      mime_type = 'application/octet-stream'
  files=[
    (
        'files',
        (
          os.path.basename(resume_path),
          open(resume_path,'rb'),
          mime_type
        )
        ),
    ]
  headers = {}
  url = urls["upload_resume"] + f"?client_id={client_id}"
  for i in range(max_tries):
    response = requests.request("POST", url, headers=headers, data=payload, files=files)
    if response.status_code == 200:
      return True, response.json()
    time.sleep(1)
    print("Retrying ", resume_path)
  print(f"Error uploading {resume_path}", response.status_code, mime_type )
  return False, response.status_code


In [5]:
def upload_jobs(job_path, max_tries = 10):
  payload={}
  mime_type, _ = mimetypes.guess_type(job_path)

  # If the MIME type cannot be determined, set it to 'application/octet-stream'
  if mime_type is None:
      mime_type = 'application/octet-stream'
  files=[
    (
        'files',
        (
          os.path.basename(job_path),
          open(job_path,'rb'),
          mime_type
        )
        ),
    ]
  headers = {}
  url = urls["upload_jobs"] 
  for i in range(max_tries):
    response = requests.request("POST", url, headers=headers, data=payload, files=files)
    if response.status_code == 200:
      return True, response.json()
    time.sleep(1)
    print("Retrying ", job_path)
  print(f"Error uploading {job_path}", response.status_code, mime_type)
  return False, response.status_code

In [6]:
def get_similarity_ranking(client_id, index = 0 ):
  url = f"https://avermass-cv-ranking.com/similarity-rankings?index={index}&client_id={client_id}"

  payload={'index': '0'}
  files=[

  ]
  headers = {}

  response = requests.request("POST", url, headers=headers, data=payload, files=files)
  if response.status_code == 200:
    return response.json()
  return False, response.status_code

In [7]:
df = []
for job in [os.listdir(job_dir)[0]]:
    job_path = os.path.join(
            os.getcwd(),
            job_dir,
            job
        )
    for resume in os.listdir(resume_dir):

        resume_path = os.path.join(
            os.getcwd(),
            resume_dir,
            resume
        )


        to_append = {
                "jobs" : job_path,
                "Resume File" : resume,
                "Status" : False,
                "similariy_ranking_response" : "",
                "client_id" : "",
                "jobs_upload_response" : "",
                "resume_upload_response"  : ""
            }
        _, upload_jobs_response = upload_jobs(job_path)
        to_append["jobs_upload_response"] = str(upload_jobs_response)
        if _:
            client_id = upload_jobs_response["client_id"]
            to_append["client_id"] = client_id
            _, upload_resume_response = upload_resume(client_id, resume_path, 10)
            to_append["resume_upload_response"] = str(upload_resume_response)
            if _:
                _, results = get_similarity_ranking(client_id=client_id)
                to_append["similariy_ranking_response"]  = str(results)
                if _:
                    to_append["Status"] =True
        print(to_append)
        df.append(to_append)


{'jobs': '/home/vk001716/ResumeRanking-NLP/Tests/JobDesc/12.docx', 'Resume File': '67.pdf', 'Status': False, 'similariy_ranking_response': '500', 'client_id': '672153f7-6c69-4f7d-8ede-30093749c944', 'jobs_upload_response': "{'file_names': ['12.docx'], 'client_id': '672153f7-6c69-4f7d-8ede-30093749c944', 'unprocessed_files': []}", 'resume_upload_response': "{'file_names': ['67.docx'], 'client_id': '672153f7-6c69-4f7d-8ede-30093749c944', 'unprocessed_files': []}"}
{'jobs': '/home/vk001716/ResumeRanking-NLP/Tests/JobDesc/12.docx', 'Resume File': '59.docx', 'Status': False, 'similariy_ranking_response': '504', 'client_id': '84da6f26-2c21-43dc-83c7-f74f540cbe3a', 'jobs_upload_response': "{'file_names': ['12.docx'], 'client_id': '84da6f26-2c21-43dc-83c7-f74f540cbe3a', 'unprocessed_files': []}", 'resume_upload_response': "{'file_names': ['59.docx'], 'client_id': '84da6f26-2c21-43dc-83c7-f74f540cbe3a', 'unprocessed_files': []}"}
Retrying  /home/vk001716/ResumeRanking-NLP/Tests/JobDesc/12.docx


In [ ]:
df_ = pd.DataFrame(df)
df_.to_csv("Test_results.csv")

In [ ]:
df_[df_.Status == False].to_csv("failed_results.csv")